# Lab: Building a Deep Research Agent with Web Search

This lab demonstrates a classic and highly valuable agentic use case: performing deep research on a given topic. We will construct a multi-agent workflow where different agents collaborate to plan search queries, execute them using a web search tool, and synthesize the findings into a comprehensive report.

### OpenAI's Hosted Tools: WebSearchTool

The OpenAI Agents SDK provides several pre-built, hosted tools that agents can use out of the box. In this lab, we will use the `WebSearchTool`, which allows an agent to search the web to find up-to-date information.

**Note on Cost**: Using the `WebSearchTool` incurs a small cost per call (check the [OpenAI pricing page](https://platform.openai.com/docs/pricing) for current rates). If cost is a concern, you can review the code without running the cells that execute the search.

In [ ]:
# === Imports ===
import os
import asyncio
from typing import Dict
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

# Core SDK components
from agents import Agent, Runner, trace, function_tool

# The built-in tool for web searches
from agents import WebSearchTool

# SendGrid for our email tool
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [ ]:
load_dotenv(override=True)

### The Research Workflow

Our system will consist of four main agents, each with a specialized role:
1.  **Planner Agent**: Creates a strategic plan of what to search for.
2.  **Search Agent**: Executes a single web search based on a query.
3.  **Writer Agent**: Synthesizes the search results into a detailed report.
4.  **Email Agent**: Formats and sends the final report via email.

#### Agent 1: The Planner
This agent takes the user's initial query and breaks it down into a series of specific, targeted search terms. We use a Pydantic model (`WebSearchPlan`) to ensure its output is a structured list of searches.

In [ ]:
# === Define Structured Output for the Planner ===
class WebSearchItem(BaseModel):
    reason: str = Field(description="A brief justification for why this specific search is necessary to answer the overall query.")
    query: str = Field(description="The exact search term to be used for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web search items that constitute the research plan.")

# === Define the Planner Agent ===
PLANNER_INSTRUCTIONS = """You are a helpful research assistant. Given a user's query, you will generate a research plan consisting of several web searches to perform. 
Your goal is to create a set of diverse and comprehensive search queries that will gather all the necessary information to answer the user's query fully."""

planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan, # Enforce structured output
)

#### Agent 2: The Searcher
This agent's sole job is to take a single search query and use the `WebSearchTool` to find relevant information on the web.

In [ ]:
# === Define the Search Agent ===
SEARCH_INSTRUCTIONS = """You are a research assistant. You will be given a search term. Your task is to use the web search tool to find information on that term and produce a concise summary of the results. 
The summary should be a few paragraphs long and capture the main points succinctly. Ignore any fluff or irrelevant details."""

search_agent = Agent(
    name="SearchAgent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")], # Provide the agent with the web search tool
    model="gpt-4o-mini",
)

#### Agent 3: The Writer
This agent receives the collected research from all the searches and synthesizes it into a single, cohesive, and detailed report. Its output is also structured to include a summary and follow-up questions.

In [ ]:
# === Define Structured Output for the Writer ===
class ReportData(BaseModel):
    short_summary: str = Field(description="A brief, 2-3 sentence executive summary of the report's findings.")
    markdown_report: str = Field(description="The final, detailed report in Markdown format. Should be at least 1000 words.")
    follow_up_questions: list[str] = Field(description="A list of suggested topics or questions for further research.")

# === Define the Writer Agent ===
WRITER_INSTRUCTIONS = """You are a senior researcher tasked with writing a comprehensive report based on a user's query and a collection of research summaries. 
First, create a clear outline. Then, using the provided research, write a detailed, well-structured report in Markdown format. 
The report should be thorough, aiming for at least 1000 words."""

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o", # Use a more powerful model for high-quality writing
    output_type=ReportData,
)

#### Agent 4: The Emailer
This final agent takes the completed report and uses a `send_email` tool to mail it.

In [ ]:
# === Define the Email Tool and Agent ===
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Sends an email with the given subject and HTML body."""
    FROM_EMAIL = "your-verified-sender@example.com"
    TO_EMAIL = "your-recipient@example.com"
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    mail = Mail(Email(FROM_EMAIL), To(TO_EMAIL), subject, Content("text/html", html_body))
    sg.client.mail.send.post(request_body=mail.get())
    return {"status": "success"}

EMAIL_INSTRUCTIONS = """You are an administrative assistant. You will be provided with a detailed report. Your task is to send this report in a nicely formatted HTML email with an appropriate subject line."""

email_agent = Agent(
    name="EmailAgent",
    instructions=EMAIL_INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

### Orchestrating the Workflow

Now we create a set of `async` functions to manage the flow of information between our agents. This is the "Plan and Execute" pattern in action.

In [ ]:
# === Define the Orchestration Logic ===

async def plan_searches(query: str) -> WebSearchPlan:
    """Step 1: Use the planner_agent to create a research plan."""
    print("--- 1. Planning searches... ---")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Plan complete. Will perform {len(result.final_output.searches)} searches.")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan) -> list[str]:
    """Step 2: Execute the search plan in parallel."""
    print("--- 2. Performing searches in parallel... ---")
    # Create a list of tasks to run concurrently
    tasks = [asyncio.create_task(search_item(item)) for item in search_plan.searches]
    # Wait for all search tasks to complete
    results = await asyncio.gather(*tasks)
    print("--- All searches complete. ---")
    return results

async def search_item(item: WebSearchItem) -> str:
    """Helper function to run a single search with the search_agent."""
    input_prompt = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input_prompt)
    return result.final_output

async def write_report(query: str, search_results: list[str]) -> ReportData:
    """Step 3: Use the writer_agent to synthesize the results into a report."""
    print("--- 3. Writing detailed report... ---")
    input_prompt = f"Original query: {query}\n\nSummarized search results:\n{'\n---\n'.join(search_results)}"
    result = await Runner.run(writer_agent, input_prompt)
    print("--- Report finished. ---")
    return result.final_output

async def email_report(report: ReportData):
    """Step 4: Use the email_agent to send the final report."""
    print("--- 4. Emailing report... ---")
    # The email agent needs a prompt to tell it what to do with the report.
    email_prompt = f"Please send the following report. The subject should be 'Research Report: {report.short_summary}'.\n\n{report.markdown_report}"
    await Runner.run(email_agent, email_prompt)
    print("--- Email sent. ---")

### Showtime! Running the Full Research Workflow

Finally, we'll execute the entire chain of functions to perform the research and get our report.

In [ ]:
# === Execute the Full Workflow ===
query = "What are the latest trends and frameworks in Agentic AI as of mid-2025?"

async def main():
    with trace("Deep_Research_Workflow"):
        print(f"--- Starting research for query: '{query}' ---")
        search_plan = await plan_searches(query)
        search_results = await perform_searches(search_plan)
        report = await write_report(query, search_results)
        await email_report(report)  
        print("\n--- Workflow Complete! ---")
        display(Markdown(f"## Research Report: {report.short_summary}"))
        display(Markdown(report.markdown_report))

# Run the main asynchronous function
await main()

### Final Check

Remember to check two places:

1.  **The Trace**: [https://platform.openai.com/traces](https://platform.openai.com/traces) to see the full, step-by-step execution of the agent team.
2.  **Your Email Inbox**: To see the final, formatted email containing the report!